In [33]:
import io
import numpy as np

def load_vectors(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    data = {}
    for line in fin:
        tokens = line.rstrip().split(' ')
        data[tokens[0]] = np.array(list(map(float, tokens[1:])))
    return data

In [34]:
embd = load_vectors('data/wiki.en.vec')

In [9]:
vecQ1 = np.zeros((3, 300), dtype = np.float32)

In [36]:
vecQ1[0,:] = embd['far']

In [37]:
embd['far']

array([-0.18551  ,  0.015872 , -0.039883 ,  0.18648  ,  0.043087 ,
        0.18006  ,  0.1393   , -0.053945 , -0.064611 ,  0.21688  ,
       -0.018256 , -0.080848 , -0.23366  , -0.27057  ,  0.084884 ,
       -0.21269  , -0.071294 ,  0.11778  ,  0.046965 ,  0.018339 ,
        0.085179 , -0.023397 , -0.15475  , -0.24089  , -0.062164 ,
       -0.087426 ,  0.2149   ,  0.015881 , -0.016373 ,  0.13569  ,
       -0.039262 ,  0.41436  , -0.43571  ,  0.4263   ,  0.087347 ,
       -0.18151  ,  0.025339 , -0.15018  , -0.23302  , -0.19211  ,
        0.03098  , -0.014375 , -0.044042 , -0.083952 , -0.050248 ,
        0.22082  ,  0.012041 , -0.12963  ,  0.076374 , -0.33944  ,
       -0.13269  , -0.21237  , -0.010519 , -0.10086  , -0.23662  ,
       -0.035792 , -0.21752  ,  0.21943  , -0.25363  ,  0.29082  ,
       -0.44182  , -0.046488 ,  0.12439  ,  0.010365 , -0.19277  ,
        0.074712 , -0.17948  ,  0.12401  ,  0.055139 ,  0.085494 ,
       -0.036012 , -0.0068908,  0.0022525, -0.24754  , -0.1868

In [39]:
'wiki.en.bin'.split('.')[-1]

'bin'

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [24]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)  # 2 words in vocab, 5 dimensional embeddings
lookup_tensor = torch.tensor([0,1,0,1,1,1], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

tensor([[-2.5667, -1.4303,  0.5009,  0.5438, -0.4057],
        [ 1.1341, -1.1115,  0.3501, -0.7703, -0.1473],
        [-2.5667, -1.4303,  0.5009,  0.5438, -0.4057],
        [ 1.1341, -1.1115,  0.3501, -0.7703, -0.1473],
        [ 1.1341, -1.1115,  0.3501, -0.7703, -0.1473],
        [ 1.1341, -1.1115,  0.3501, -0.7703, -0.1473]],
       grad_fn=<EmbeddingBackward>)


In [43]:
for i in embeds.parameters():
    print(i)

Parameter containing:
tensor([[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.1341, -1.1115,  0.3501, -0.7703, -0.1473]], grad_fn=<CopySlices>)


In [58]:
from torch.autograd import Variable
a = Variable(torch.randn(3,2), requires_grad=True)
a

tensor([[ 0.6984, -0.8005],
        [ 1.5381,  1.4673],
        [ 1.5951, -1.5279]], requires_grad=True)

False

In [59]:
b = torch.randn(3,2,5)

In [61]:
b[0,0,].size()

torch.Size([5])

In [63]:
a = torch.randn(2,3,4)
b = torch.randn(1,4,3)
torch.matmul(a, b).size()

torch.Size([2, 3, 3])

In [66]:
a.repeat(1,2,1).size()

torch.Size([2, 6, 4])

In [71]:
a = torch.randn(1, 3, 2) # [B, T, D]
a

tensor([[[ 0.8803,  0.5456],
         [-0.5979,  0.3257],
         [-0.5642, -1.1759]]])

In [77]:
a = a.repeat(1, 1, 3).view(1, 3, 3, 2) # [B, T, T, D]
a

tensor([[[[ 0.8803,  0.5456],
          [ 0.8803,  0.5456],
          [ 0.8803,  0.5456]],

         [[-0.5979,  0.3257],
          [-0.5979,  0.3257],
          [-0.5979,  0.3257]],

         [[-0.5642, -1.1759],
          [-0.5642, -1.1759],
          [-0.5642, -1.1759]]]])

In [87]:
h = torch.randn(1, 3, 4, 2) # [B, T, T2, D]
h

tensor([[[[ 1.5083,  0.1054],
          [-1.6050, -0.1064],
          [ 0.2466,  0.6125],
          [-0.0077,  2.6158]],

         [[ 1.0757, -0.5536],
          [-1.6160,  0.0934],
          [-1.3898, -0.3105],
          [ 1.0693,  1.4394]],

         [[ 1.3694,  0.4539],
          [-0.0498,  0.3745],
          [ 1.4389,  1.4151],
          [-0.1589, -0.7360]]]])

In [88]:
result = torch.diagonal( a.matmul(h.transpose(-2, -1)), dim1=1, dim2=2 ).transpose(-2, -1)
result

tensor([[[ 1.3853, -1.4709,  0.5513,  1.4205],
         [-0.8234,  0.9966,  0.7298, -0.1705],
         [-1.3064, -0.4123, -2.4758,  0.9552]]])

In [86]:
result.size() # [B, T, T2]

torch.Size([1, 3, 4])

In [89]:
D = torch.randn(2, 5, 5)
torch.diagonal(D, dim1=1, dim2=2).size()

torch.Size([2, 5])